# Packages Installation

In [ ]:
import pandas as pd
import re
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

# Load dataset

In [ ]:
!pip install datasets

from datasets import load_dataset

In [ ]:
dataset = load_dataset("Johnnyeee/Yelpdata_663",split=['train[:1%]'], trust_remote_code=True)

/Users/johnnys/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
dataset

[Dataset({
     features: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_x', 'review_count', 'is_open', 'categories', 'hours', 'review_id', 'user_id', 'stars_y', 'useful', 'funny', 'cool', 'text', 'date', 'attributes'],
     num_rows: 37787
 })]

In [ ]:
train_df = pd.DataFrame(dataset[0])

In [ ]:
train_df = train_df[:10001]

In [ ]:
train_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,hours,review_id,user_id,stars_y,useful,funny,cool,text,date,attributes
0,XQfwVwDr-v0ZS3_CbbE5Xw,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210197,-75.223640,3.0,169.0,...,"{""Monday"": ""7:30-15:0"", ""Tuesday"": ""7:30-15:0""...",KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"{""NoiseLevel"": ""u'average'"", ""HasTV"": ""False"",..."
1,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,748 W Starr Pass Blvd,Tucson,AZ,85713,32.207233,-110.980865,3.5,47.0,...,null,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,3.0,0.0,0.0,0.0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,"{""RestaurantsReservations"": ""True"", ""BusinessP..."
2,kxX2SOes4o-D3ZQBkiMRfA,Zaika,2481 Grant Ave,Philadelphia,PA,19114,40.079849,-75.025078,4.0,181.0,...,"{""Tuesday"": ""11:0-21:0"", ""Wednesday"": ""11:0-21...",AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,"{""Caters"": ""True"", ""Ambience"": ""{'romantic': F..."
3,e4Vwtrqf-wpJfwesgvdgxQ,Melt,2549 Banks St,New Orleans,LA,70119,29.962103,-90.087959,4.0,32.0,...,"{""Monday"": ""0:0-0:0"", ""Friday"": ""11:0-17:0"", ""...",Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,"{""BusinessParking"": ""{'garage': False, 'street..."
4,04UD14gamNjLY0IDYVhHJg,Dmitri's,795 S 3rd St,Philadelphia,PA,19147,39.938015,-75.148132,4.0,273.0,...,"{""Wednesday"": ""17:30-21:0"", ""Thursday"": ""17:30...",JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,"{""BusinessParking"": ""{'garage': False, 'street..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,pSmOH4a3HNNpYM82J5ycLA,The Pancake Pantry,1796 21st Ave S,Nashville,TN,37212,36.136986,-86.800537,4.0,2063.0,...,"{""Monday"": ""0:0-0:0"", ""Tuesday"": ""6:0-15:0"", ""...",y7jQXGtZF-iZ0OFaZ9n-eA,esXFBGjptPHN9figivJq_Q,5.0,0.0,0.0,0.0,If you like pancakes and efficient no frill se...,2015-05-12 16:46:38,"{""RestaurantsTakeOut"": ""True"", ""Corkage"": ""Fal..."
9997,g04aAvgol7IW8buqSbT4xA,Cafe Fleur De Lis,307 Chartres St,New Orleans,LA,70116,29.954494,-90.066589,4.0,1865.0,...,"{""Monday"": ""7:0-13:0"", ""Tuesday"": ""7:0-13:0"", ...",Q2EelDC5upkqmNL_e3kibA,f9WX3CCzChvsuvI2rlt1LA,5.0,1.0,0.0,0.0,Visited New Orleans for the end of Mardi Gras ...,2017-03-03 19:56:51,"{""RestaurantsReservations"": ""False"", ""Alcohol""..."
9998,f4vbnGoGo3eWorVekctVGQ,Refuel Cafe,8124 Hampson St,New Orleans,LA,70118,29.944195,-90.134163,4.5,328.0,...,"{""Monday"": ""7:0-11:0"", ""Tuesday"": ""7:0-14:0"", ...",TpiEUZVA6rGKFuYpvA_aRQ,vbKjo1oGoTdhMrIB9AKVLQ,5.0,0.0,0.0,0.0,This is one of my favorite little neighborhood...,2014-02-24 02:31:57,"{""WiFi"": ""'free'"", ""RestaurantsAttire"": ""'casu..."
9999,JF7iyI3wA_AKi_iXOKi0Qw,Parlor Suburban Square,30 Parking Plz,Ardmore,PA,19003,40.008644,-75.288803,3.5,61.0,...,"{""Monday"": ""11:30-22:0"", ""Tuesday"": ""11:30-22:...",-Ub4MvvC960ud5wqQFErVA,d60qj9UKo4SWOK7TR24dtg,1.0,0.0,0.0,0.0,The only good thing about this place is they k...,2015-05-12 17:45:17,"{""Music"": ""{'live': False}"", ""Ambience"": ""{'ro..."


# Data preprocessing

In [ ]:
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK's stopwords list

import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Get the English stopwords list and convert to lowercase
stop_words = set(word.lower() for word in stopwords.words("english"))

# Ensure that 'not' and contractions ending with 'n't' are not treated as stopwords
stop_words = stop_words - set(['not', "no", "isn't", "aren't", "wasn't", "weren't", "haven't", "hasn't", "hadn't",
                               "doesn't", "don't", "didn't", "won't", "wouldn't", "shan't", "shouldn't", "can't",
                               "cannot", "couldn't", "mustn't", "mightn't"])

# Tokenize each text in data["text"], convert to lowercase, and remove stopwords
filtered_texts = []
for text in train_df["text"]:
    # Convert to lowercase and tokenize the text
    words = word_tokenize(text.lower())  # Convert to lowercase here
    # Filter out stopwords, preserving 'not' and "n't"
    filtered_words = [word for word in words if word not in stop_words]
    # Join the filtered words back into a sentence
    filtered_text = " ".join(filtered_words)
    # Append the filtered text to the list
    filtered_texts.append(filtered_text)

# Replace the original text data with the filtered texts
train_df["filter_text"] = filtered_texts



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/johnnys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/johnnys/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re

def remove_punctuation(text):
    # Define the regular expression pattern to match punctuation and ellipses
    # but not the apostrophe in contractions such as n't
    pattern = r"(?<!\w)['\"](?!\w)|[\.,!?;:()\-—\[\]{}<>@#%^&*+=|\\/~`…]"
    # Replace punctuation, ellipses, and parentheses with an empty string
    return re.sub(pattern, '', text)

# Apply the function to the "filter_text" column
train_df["punctuation"] = train_df["filter_text"].apply(remove_punctuation)


In [ ]:
import re

def remove_non_english(text):
    # Define a pattern to match non-ASCII characters (foreign language characters)
    pattern = '[^\x00-\x7F]+'
    # Replace non-ASCII characters with an empty string
    return re.sub(pattern, '', text)

# Apply the function to the "filter_text" column to remove non-English text
train_df["foreign"] = train_df["punctuation"].apply(remove_non_english)


In [ ]:
# Split the text in the "punctuation" column and count the occurrences of each word
word_counts = pd.Series(" ".join(train_df["foreign"]).split()).value_counts()

other_words_h = ["food", "place", "restaurant", "get", "go", "went", "come", "came", "us", "bar", "also", "try", "ve", "s"]
other_words_l = word_counts[word_counts == 1].index.tolist()

# Combine the lists using list concatenation
other_words = other_words_l + other_words_h

In [ ]:
# Assuming other_words is a list, convert it to a set for faster lookup
other_words_set = set(other_words)

# A more efficient approach to remove unwanted words
def remove_unwanted_words(text):
    return " ".join(word for word in text.split() if word not in other_words_set)

# Apply the function to the 'foreign' column
# This uses the map function which can be faster than apply+lambda on some occasions
train_df["clean"] = train_df["foreign"].map(remove_unwanted_words)

In [ ]:
train_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,useful,funny,cool,text,date,attributes,filter_text,punctuation,foreign,clean
0,XQfwVwDr-v0ZS3_CbbE5Xw,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210197,-75.223640,3.0,169.0,...,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"{""NoiseLevel"": ""u'average'"", ""HasTV"": ""False"",...","decide eat , aware going take 2 hours beginnin...",decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning ...
1,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,748 W Starr Pass Blvd,Tucson,AZ,85713,32.207233,-110.980865,3.5,47.0,...,0.0,0.0,0.0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,"{""RestaurantsReservations"": ""True"", ""BusinessP...",family diner . buffet . eclectic assortment : ...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment large ...
2,kxX2SOes4o-D3ZQBkiMRfA,Zaika,2481 Grant Ave,Philadelphia,PA,19114,40.079849,-75.025078,4.0,181.0,...,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,"{""Caters"": ""True"", ""Ambience"": ""{'romantic': F...","wow ! yummy , different , delicious . favorite...",wow yummy different delicious favorite lam...,wow yummy different delicious favorite lam...,wow yummy different delicious favorite lamb cu...
3,e4Vwtrqf-wpJfwesgvdgxQ,Melt,2549 Banks St,New Orleans,LA,70119,29.962103,-90.087959,4.0,32.0,...,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,"{""BusinessParking"": ""{'garage': False, 'street...",cute interior owner ( ? ) gave us tour upcomin...,cute interior owner gave us tour upcoming p...,cute interior owner gave us tour upcoming p...,cute interior owner gave tour upcoming area gr...
4,04UD14gamNjLY0IDYVhHJg,Dmitri's,795 S 3rd St,Philadelphia,PA,19147,39.938015,-75.148132,4.0,273.0,...,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,"{""BusinessParking"": ""{'garage': False, 'street...",long term frequent customer establishment . we...,long term frequent customer establishment wen...,long term frequent customer establishment wen...,long term frequent customer establishment orde...


# Sentiment Analysis

In [ ]:
!pip install nltk textblob
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /Users/johnnys/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
import nltk

# Ensure necessary NLTK downloads are complete
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to lemmatize a sentence
def lemmatize_sentence(sentence):
    words = sentence.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

# Lemmatize the "clean" column
train_df["lemmatized"] = train_df["clean"].apply(lemmatize_sentence)

# Perform sentiment analysis and store the results in two new columns
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment

train_df[['polarity', 'subjectivity']] = train_df['lemmatized'].apply(get_sentiment).tolist()

[nltk_data] Downloading package wordnet to /Users/johnnys/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/johnnys/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/Users/johnnys/anaconda3/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
def polarity_scores_roberta(text):
    # Encode the text, ensuring that the sequence is truncated/padded to a length of 512 tokens
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

    outputs = model(**inputs)

    scores = outputs[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/johnnys/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Pre-define the dictionary to store results
res = {}

# Iterate over the DataFrame with tqdm for a progress bar
for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    try:
        # Extract text and review_id from the row
        text = row['text']
        myid = row['review_id']

        # Calculate VADER sentiment scores
        vader_result = sia.polarity_scores(text)

        # Calculate RoBERTa sentiment scores (function not shown in the image)
        roberta_result = polarity_scores_roberta(text)

        # Calculate a combined polarity score (assuming the RoBERTa function returns a dict with these keys)
        roberta_polarity_score = -1 * roberta_result['roberta_neg'] + roberta_result['roberta_pos']

        # Store the combined score in the result dictionary using review_id as the key
        res[myid] = roberta_polarity_score

    except Exception as e:  # Catch any exception and print the error along with the id
        print(f'Error for id {myid}: {e}')

# After the loop, you may want to convert the results dictionary back to a DataFrame
result_df = pd.DataFrame.from_dict(res, orient='index', columns=['roberta_polarity_score'])

  0%|          | 0/10001 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# Let's first create a DataFrame from the 'res' dictionary which we assume is structured as shown in the image
res_df = pd.DataFrame(list(res.items()), columns=['review_id', 'roberta_polarity'])

# Now let's assume 'df' is your original DataFrame and has a 'review_id' column
# We will merge 'res_df' with 'df' based on the 'review_id' column
final_df = train_df.merge(res_df, on='review_id', how='left')

# 'final_df' now contains the original data along with the 'roberta_polarity' from 'res'


In [ ]:
final_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,date,attributes,filter_text,punctuation,foreign,clean,lemmatized,polarity,subjectivity,roberta_polarity
0,XQfwVwDr-v0ZS3_CbbE5Xw,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210197,-75.223640,3.0,169.0,...,2018-07-07 22:09:11,"{""NoiseLevel"": ""u'average'"", ""HasTV"": ""False"",...","decide eat , aware going take 2 hours beginnin...",decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning ...,decide eat aware going take 2 hour beginning e...,0.198485,0.412121,0.656539
1,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,748 W Starr Pass Blvd,Tucson,AZ,85713,32.207233,-110.980865,3.5,47.0,...,2014-02-05 20:30:30,"{""RestaurantsReservations"": ""True"", ""BusinessP...",family diner . buffet . eclectic assortment : ...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment large ...,family diner buffet eclectic assortment large ...,0.139935,0.522294,0.961204
2,kxX2SOes4o-D3ZQBkiMRfA,Zaika,2481 Grant Ave,Philadelphia,PA,19114,40.079849,-75.025078,4.0,181.0,...,2015-01-04 00:01:03,"{""Caters"": ""True"", ""Ambience"": ""{'romantic': F...","wow ! yummy , different , delicious . favorite...",wow yummy different delicious favorite lam...,wow yummy different delicious favorite lam...,wow yummy different delicious favorite lamb cu...,wow yummy different delicious favorite lamb cu...,0.315152,0.733838,0.984615
3,e4Vwtrqf-wpJfwesgvdgxQ,Melt,2549 Banks St,New Orleans,LA,70119,29.962103,-90.087959,4.0,32.0,...,2017-01-14 20:54:15,"{""BusinessParking"": ""{'garage': False, 'street...",cute interior owner ( ? ) gave us tour upcomin...,cute interior owner gave us tour upcoming p...,cute interior owner gave us tour upcoming p...,cute interior owner gave tour upcoming area gr...,cute interior owner gave tour upcoming area gr...,0.416234,0.490260,0.980289
4,04UD14gamNjLY0IDYVhHJg,Dmitri's,795 S 3rd St,Philadelphia,PA,19147,39.938015,-75.148132,4.0,273.0,...,2015-09-23 23:10:31,"{""BusinessParking"": ""{'garage': False, 'street...",long term frequent customer establishment . we...,long term frequent customer establishment wen...,long term frequent customer establishment wen...,long term frequent customer establishment orde...,long term frequent customer establishment orde...,0.211111,0.405556,-0.901215


## the difference between stars_y and polarity

In [ ]:
def standardize_range(values, min_original, max_original, min_new=-1, max_new=1):
    return [(max_new - min_new) * (value - min_original) / (max_original - min_original) + min_new for value in values]


standardized_values = standardize_range(final_df['stars_y'], min(final_df['stars_y']), max(final_df['stars_y']))


In [ ]:
final_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,date,attributes,filter_text,punctuation,foreign,clean,lemmatized,polarity,subjectivity,roberta_polarity
0,XQfwVwDr-v0ZS3_CbbE5Xw,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210197,-75.223640,3.0,169.0,...,2018-07-07 22:09:11,"{""NoiseLevel"": ""u'average'"", ""HasTV"": ""False"",...","decide eat , aware going take 2 hours beginnin...",decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning...,decide eat aware going take 2 hours beginning ...,decide eat aware going take 2 hour beginning e...,0.198485,0.412121,0.656539
1,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,748 W Starr Pass Blvd,Tucson,AZ,85713,32.207233,-110.980865,3.5,47.0,...,2014-02-05 20:30:30,"{""RestaurantsReservations"": ""True"", ""BusinessP...",family diner . buffet . eclectic assortment : ...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment lar...,family diner buffet eclectic assortment large ...,family diner buffet eclectic assortment large ...,0.139935,0.522294,0.961204
2,kxX2SOes4o-D3ZQBkiMRfA,Zaika,2481 Grant Ave,Philadelphia,PA,19114,40.079849,-75.025078,4.0,181.0,...,2015-01-04 00:01:03,"{""Caters"": ""True"", ""Ambience"": ""{'romantic': F...","wow ! yummy , different , delicious . favorite...",wow yummy different delicious favorite lam...,wow yummy different delicious favorite lam...,wow yummy different delicious favorite lamb cu...,wow yummy different delicious favorite lamb cu...,0.315152,0.733838,0.984615
3,e4Vwtrqf-wpJfwesgvdgxQ,Melt,2549 Banks St,New Orleans,LA,70119,29.962103,-90.087959,4.0,32.0,...,2017-01-14 20:54:15,"{""BusinessParking"": ""{'garage': False, 'street...",cute interior owner ( ? ) gave us tour upcomin...,cute interior owner gave us tour upcoming p...,cute interior owner gave us tour upcoming p...,cute interior owner gave tour upcoming area gr...,cute interior owner gave tour upcoming area gr...,0.416234,0.490260,0.980289
4,04UD14gamNjLY0IDYVhHJg,Dmitri's,795 S 3rd St,Philadelphia,PA,19147,39.938015,-75.148132,4.0,273.0,...,2015-09-23 23:10:31,"{""BusinessParking"": ""{'garage': False, 'street...",long term frequent customer establishment . we...,long term frequent customer establishment wen...,long term frequent customer establishment wen...,long term frequent customer establishment orde...,long term frequent customer establishment orde...,0.211111,0.405556,-0.901215


In [ ]:
final_df['standard_stars_y'] = standardized_values

In [ ]:
final_df['absolute_diff'] = abs(final_df['roberta_polarity']-final_df['standard_stars_y'])

## word count

In [ ]:
final_df['word_count'] = final_df['lemmatized'].apply(lambda x: len(x.split()))

## funny/cool/useful count

In [ ]:
final_df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_x', 'review_count', 'is_open',
       'categories', 'hours', 'review_id', 'user_id', 'stars_y', 'useful',
       'funny', 'cool', 'text', 'date', 'attributes', 'filter_text',
       'punctuation', 'foreign', 'clean', 'lemmatized', 'polarity',
       'subjectivity', 'roberta_polarity', 'standard_stars_y', 'absolute_diff',
       'word_count'],
      dtype='object')

In [ ]:
final_df['FCU_count'] = final_df['useful']+final_df['funny']+final_df['cool']

# Final Output

In [ ]:
final_df.to_csv('output.csv', index=False)